# Installation
Just pip install:

```pip install omegaconf```

If you want to try this notebook after checking out the repository be sure to run 
```python setup.py develop``` at the repository root before running this code.

# Creating OmegaConf objects
### Empty

In [1]:
from omegaconf import OmegaConf
conf = OmegaConf.create()
print(conf)

{}


### From a dictionary

In [2]:
conf = OmegaConf.create(dict(k='v',list=[1,dict(a='1',b='2')]))
print(conf.pretty())

k: v
list:
- 1
- a: '1'
  b: '2'



### From a list

In [3]:
conf = OmegaConf.create([1, dict(a=10, b=dict(a=10))])
print(conf.pretty())

- 1
- a: 10
  b:
    a: 10



### From a yaml file

In [4]:
conf = OmegaConf.load('../source/example.yaml')
print(conf.pretty())

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



### From a yaml string

In [5]:
yaml = """
a: b
b: c
list:
- item1
- item2
"""
conf = OmegaConf.create(yaml)
print(conf.pretty())

a: b
b: c
list:
- item1
- item2



### From a dot-list

In [6]:
dot_list = ["a.aa.aaa=1", "a.aa.bbb=2", "a.bb.aaa=3", "a.bb.bbb=4"]
conf = OmegaConf.from_dotlist(dot_list)
print(conf.pretty())

a:
  aa:
    aaa: 1
    bbb: 2
  bb:
    aaa: 3
    bbb: 4



### From command line arguments

To parse the content of sys.arg:

In [7]:
# Simulating command line arguments
import sys
sys.argv = ['your-program.py', 'server.port=82', 'log.file=log2.txt']
conf = OmegaConf.from_cli()
print(conf.pretty())

server:
  port: 82
log:
  file: log2.txt



# Access and manipulation
Input yaml file:

In [8]:
conf = OmegaConf.load('../source/example.yaml')
print(conf.pretty())

server:
  port: 80
log:
  file: ???
  rotation: 3600
users:
- user1
- user2



#### Object style access:

In [9]:
conf.server.port

80

#### dictionary style access

In [10]:
conf['log']['rotation']

3600

#### items in list

In [11]:
conf.users[0]

'user1'

#### Changing existing keys

In [12]:
conf.server.port = 81

#### Adding new keys

In [13]:
conf.server.hostname = "localhost"

#### Adding a new dictionary

In [14]:
conf.database = {'hostname': 'database01', 'port': 3306}

#### providing default values

In [15]:
conf.missing_key or 'a default value'
conf.get('missing_key', 'a default value')

'a default value'

#### Accessing mandatory values
Accessing fields with the value *???* will cause a MissingMandatoryValue exception.
Use this to indicate that the value must be set before accessing.

In [16]:
import pytest
from omegaconf import MissingMandatoryValue

with pytest.raises(MissingMandatoryValue):
    conf.log.file

# Variable interpolation

OmegaConf support variable interpolation, Interpolations are evaluated lazily on access.

### Config node interpolation

Interpolations are evaluated lazily on field access.<br/>
Note below that when printed the interpolations are not resolved.<br/>
They get resolved once you access them.

In [17]:
conf = OmegaConf.load('../source/config_interpolation.yaml')
print(conf.pretty())

server:
  host: localhost
  port: 80
client:
  url: http://${server.host}:${server.port}/
  server_port: ${server.port}



In [18]:
# Primitive interpolation types are inherited from the referenced value
print("conf.client.server_port: ", conf.client.server_port, type(conf.client.server_port).__name__)
# Composite interpolation types are always string
print("conf.client.url: ", conf.client.url, type(conf.client.url).__name__)

conf.client.server_port:  80 int
conf.client.url:  http://localhost:80/ str


You can resolve interpolation while you are printing using `resolve=True`.

In [19]:
print(conf.pretty(resolve=True))

server:
  host: localhost
  port: 80
client:
  url: http://localhost:80/
  server_port: 80



### Environment variable interpolation

Environment variable interpolation is also supported.

Input yaml file:

In [20]:
# Let's set up the environment first (Only needed for this demonstration)
import os
os.environ['USER'] = 'omry'

Here is an example config file interpolates with the USER environment variable:

In [21]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(conf.pretty())

user:
  name: ${env:USER}
  home: /home/${env:USER}



In [22]:
conf = OmegaConf.load('../source/env_interpolation.yaml')
print(conf.pretty(resolve=True))

user:
  name: omry
  home: /home/omry



You can also set a default value for environment variables:

In [23]:
conf = OmegaConf.create({"user" : {"age" : "${env:AGE, 42}"}})
print(conf.user.age)

42


## Custom interpolations

You can add additional interpolation types using custom resolvers. This example creates a resolver that adds 10 the the given value.

In [24]:
OmegaConf.register_resolver("plus_10", lambda x: int(x) + 10)
conf = OmegaConf.create({'key': '${plus_10:990}'})
print(conf.key)

1000


# Merging configurations
Merging configurations enables the creation of reusable configuration files for each logical component instead of a single config file for each variation of your task.

Machine learning experiment example:
```python
conf = OmegaConf.merge(base_cfg, model_cfg, optimizer_cfg, dataset_cfg)
```

Web server configuration example:

```python
conf = OmegaConf.merge(server_cfg, plugin1_cfg, site1_cfg, site2_cfg)
```

The following example creates two configs from files, and one from the cli. It then combines them into a single object. Note how the port changes to 82, and how the users lists are combined.

In [25]:
base_conf = OmegaConf.load('../source/example2.yaml')
print(base_conf.pretty())

server:
  port: 80
users:
- user1
- user2



In [26]:
second_conf = OmegaConf.load('../source/example3.yaml')
print(second_conf.pretty())

log:
  file: log.txt



In [27]:
from omegaconf import OmegaConf
import sys

# Merge configs:
conf = OmegaConf.merge(base_conf, second_conf)

# Simulate command line arguments
sys.argv = ['program.py', 'server.port=82']
# Merge with cli arguments
conf.merge_with_cli()
print(conf.pretty())

server:
  port: 82
users:
- user1
- user2
log:
  file: log.txt

